In [1]:
!git clone https://github.com/yaledhlab/vggface.git

fatal: destination path 'vggface' already exists and is not an empty directory.


In [2]:
cd vggface

/home/adarsh/celebrity_face_detection/vggface


In [3]:
!pwd

/home/adarsh/celebrity_face_detection/vggface


In [4]:
!python3 setup.py install

running install
/home/adarsh/.local/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/home/adarsh/.local/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.


In [4]:
!pip install keras_applications==1.0.8

Defaulting to user installation because normal site-packages is not writeable


In [5]:
from tensorflow.keras.preprocessing import image
from keras.utils import plot_model
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
import numpy as np
import pickle
from tqdm import tqdm

In [6]:
cd ..

/home/adarsh/celebrity_face_detection


In [7]:
with open('filenames.pickle','rb') as file:
    filenames = pickle.load(file)

In [8]:
model = VGGFace(model='resnet50' ,include_top=False ,input_shape=(244,244,3) ,pooling='avg')
print(model)

In [9]:
model.summary()

Model: "vggface_resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 244, 244, 3)]        0         []                            
                                                                                                  
 conv1/7x7_s2 (Conv2D)       (None, 122, 122, 64)         9408      ['input_1[0][0]']             
                                                                                                  
 conv1/7x7_s2/bn (BatchNorm  (None, 122, 122, 64)         256       ['conv1/7x7_s2[0][0]']        
 alization)                                                                                       
                                                                                                  
 activation (Activation)     (None, 122, 122, 64)         0         ['conv1/7x7_s2/

In [12]:
plot_model(model ,to_file='model_structure_plot.png' ,show_shapes=True ,show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [13]:
def feature_extractor(img_path ,model):
  img = image.load_img(img_path ,target_size=(244,244))
  img_array = image.img_to_array(img)
  expanded_img = np.expand_dims(img_array ,axis=0)
  preprocess_img = preprocess_input(expanded_img)

  result = model.predict(preprocess_img).flatten()
  return result

In [15]:
features = []
for file in filenames:
  result = feature_extractor(file ,model)
  features.append(result)

pickle.dump(features ,open('embedding.pickle','wb'))

1/1 [==============================] - 0s 189ms/step


In [18]:
len(features)

8664

In [20]:
len(features[0])

2048

In [10]:
with open('vggface.pickle','wb') as file:
    pickle.dump(model ,file)